# PySpark and Spark SQL with Tables

In this notebook we will use the Boston dataset and store it the spark datawarehouse. We will then use the datawarehouse to access this data using spark sql.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession;

spark = SparkSession.builder.master("local[4]").appName("ISM6562 Spark App01").enableHiveSupport().getOrCreate();

# Let's get the SparkContext object. It's the entry point to the Spark API. It's created when you create a sparksession
sc = spark.sparkContext  

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

Spark Session WebUI Port: 4042


In [2]:
# this will set the log level to ERROR. This will hide the INFO or WARNING messages that are printed out by default. If you want to see them, set this to INFO or WARN.
sc.setLogLevel("ERROR") 

Display the spark object - this provides the link to the Spark UI

In [3]:
spark

List the Spark datawarehouse. It should show the default database.

In [4]:
df=spark.sql("show databases")
df.show()

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

List the tables in the database. If this is a new install (you haven't run this notebook before), there shouldn't be any tables.

In [ ]:
tables = spark.sql("show tables").show()

Now, let's load the Boston dataset into the datawarehouse. We will use the spark dataframe API to load the data. We will then use the spark sql API to create a table from the dataframe.

In [ ]:
boston = spark.read.csv('BostonHousing.csv', header=True, inferSchema=True);

# display the first 5 rows of the dataframe
boston.show(5);

DataFrames can also be saved as persistent tables into Hive metastore using the saveAsTable command. Notice that an existing Hive deployment is not necessary to use this feature. Spark will create a default local Hive metastore (using Derby) for you. Unlike the createOrReplaceTempView command, saveAsTable will materialize the contents of the DataFrame and create a pointer to the data in the Hive metastore. Persistent tables will still exist even after your Spark program has restarted, as long as you maintain your connection to the same metastore. A DataFrame for a persistent table can be created by calling the table method on a SparkSession with the name of the table.

see https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#saving-to-persistent-tables

We can create a table from a dataframe using the spark sql API. This will create a table in memory. The table will be lost when the spark session is terminated.

In [ ]:
boston.createOrReplaceTempView("boston_tmp_view")

We can use SQL to query the table. The result is a dataframe.

In [ ]:
df = spark.sql("select * from boston_tmp_view")
df.show(5)

Now, when we list the tables, we should see the boston table (and the temp table we created earlier).

In [ ]:
tables = spark.sql("show tables").show()

We can now use the spark sql API to query the table.

In [ ]:
df = spark.sql("select * from boston")
df.show()

To save the table to the spark datawarehouse, we use the saveAsTable command. This will create a table in the spark datawarehouse.

In [ ]:
boston.write.saveAsTable("boston", mode='overwrite')

In [ ]:
spark.catalog.listTables()

In [ ]:
df = spark.sql("select * from boston")
df.show()

If we wish to drop a table from the warehouse, we can use the drop command.

In [ ]:
# For now, we will keep the table and access it in another notebook.
# spark.sql("DROP TABLE boston")

In [ ]:
spark.catalog.listTables()

In [9]:
# spark.stop()